# Task 3: Classification of COVID19 tweets containing symptoms

In [1]:
!pip install -q transformers contractions imbalanced-learn ekphrasis

     |████████████████████████████████| 4.0 MB 4.8 MB/s 
     |████████████████████████████████| 80 kB 2.7 MB/s 
     |████████████████████████████████| 6.6 MB 32.0 MB/s 
     |████████████████████████████████| 895 kB 35.1 MB/s 
     |████████████████████████████████| 77 kB 3.8 MB/s 
     |████████████████████████████████| 596 kB 33.6 MB/s 
     |████████████████████████████████| 106 kB 12.0 MB/s 
     |████████████████████████████████| 287 kB 32.3 MB/s 
     |████████████████████████████████| 45 kB 2.2 MB/s 
     |████████████████████████████████| 53 kB 1.4 MB/s 


## 1. Import all the necessary libraries and data files

In [2]:
import numpy as np
import pandas as pd

import warnings
import torch
import torch.nn as nn
import time

from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertModel, BertTokenizerFast
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons
from tqdm import tqdm
from sklearn.metrics import f1_score

warnings.filterwarnings("ignore")
pd.options.display.max_colwidth=None

In [3]:
train_filename = "train.tsv"
val_filename = "valid.tsv"

In [4]:
# Load data
train = pd.read_csv(train_filename, sep="\t")
validation = pd.read_csv(val_filename, sep="\t")

In [5]:
print(f"Shape of training data is {train.shape} and validation data is {validation.shape}")

Shape of training data is (9067, 3) and validation data is (500, 3)


In [6]:
# Train top 5 rows
train.head().style.set_caption("Task 3: Train dataset")

,tweet_id,tweet,label
0,13729,A growing number of Covid-19 patients whose symptoms were initially mild are now facing mysterious long-term neurological problems https://t.co/If2SgRduuw,Lit-News_mentions
1,12399,"Medical experts advise that symptoms of the novel coronavirus include fever, shortness of breath, and stinky smelly pits and feet 😳🤪",Lit-News_mentions
2,20056,"@drdavidsamadi Hubby/I:same symptoms n November 2019 after a weekend trip 2 Vegas where bus loads of Chinese tourists.1 day fever,3 days sore throat,several weeks of fatigue.He's healthy,I'm not: diabetes,hypertension,obese, respiratory issues @ 53. No meds/pneumonia,we believe was COVID-19",Nonpersonal_reports
3,10175,"1/x In the April 11 BC briefing Dr. Bonnie Henry had mentioned that there's now reports of neurologic complications after COVID-19 infection, even during recovery. There's now anecdotal reporting of neurological manifestations from WUHAN patients in JAMA:https://t.co/7spTyk7l2M",Lit-News_mentions
4,12179,Major study PHOSP-COVID investigates health impacts of #COVID19 on hospitalised patients including #mentalhealth & neurological problems. Find out more https://t.co/JLTrz0BA7f @OxfordHealthNHS https://t.co/jZ2kPyPqmS,Lit-News_mentions


## 2. Prepare the data - Clean & Prepare for Model

In [7]:
# Drop unwanted columns
train.drop(['tweet_id'], axis=1, inplace=True)
validation.drop(['tweet_id'], axis=1, inplace=True)

In [8]:
# Referred from: https://github.com/cbaziotis/ekphrasis

text_processor = TextPreProcessor(
    # terms that will be normalized
    normalize=['url', 'email', 'percent', 'money', 'phone', 'user',
        'time', 'url', 'date', 'number'],
    
    # terms that will be annotated
    annotate={"hashtag", "allcaps", "elongated", "repeated",
        'emphasis', 'censored'},
    fix_html=True,  # fix HTML tokens
    
    # corpus from which the word statistics are going to be used 
    # for word segmentation 
    segmenter="twitter", 
    
    # corpus from which the word statistics are going to be used 
    # for spell correction
    corrector="twitter", 
    
    unpack_hashtags=True,  # perform word segmentation on hashtags
    unpack_contractions=True,  # Unpack contractions (can't -> can not)
    spell_correct_elong=False,  # spell correction for elongated words
    
    # select a tokenizer. You can use SocialTokenizer, or pass your own
    # the tokenizer, should take as input a string and return a list of tokens
    tokenizer=SocialTokenizer(lowercase=True, emojis=False).tokenize,
    
    # list of dictionaries, for replacing tokens extracted from the text,
    # with other expressions. You can pass more than one dictionaries.
    dicts=[emoticons]
)

Word statistics files not found!
Downloading... done!
Unpacking... done!
Reading twitter - 1grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/twitter/counts_1grams.txt
Reading twitter - 2grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/twitter/counts_2grams.txt
Reading twitter - 1grams ...


In [9]:
train['clean_tweets'] = [" ".join(text_processor.pre_process_doc(tweet)) for tweet in train.tweet]
validation['clean_tweets'] = [" ".join(text_processor.pre_process_doc(tweet)) for tweet in validation.tweet]

In [10]:
# Train top 5 rows after pre-processing
train[['label', 'clean_tweets']].head()

,label,clean_tweets
0,Lit-News_mentions,a growing number of covid - <number> patients whose symptoms were initially mild are now facing mysterious long - term neurological problems <url>
1,Lit-News_mentions,"medical experts advise that symptoms of the novel coronavirus include fever , shortness of breath , and stinky smelly pits and feet 😳 🤪"
2,Nonpersonal_reports,"<user> hubby / i : same symptoms n <date> after a weekend trip <number> vegas where bus loads of chinese tourists . <number> day fever , <number> days sore throat , several weeks of fatigue . he ' s healthy , i am not : diabetes , hypertension , obese , respiratory issues @ <number> . no meds / pneumonia , we believe was <allcaps> covid </allcaps> - <number>"
3,Lit-News_mentions,"<number> /x in the <date> bc briefing dr . bonnie henry had mentioned that there ' s now reports of neurologic complications after <allcaps> covid </allcaps> - <number> infection , even during recovery . there ' s now anecdotal reporting of neurological manifestations from <allcaps> wuhan </allcaps> patients in <allcaps> jama </allcaps> : <url>"
4,Lit-News_mentions,major study <allcaps> phosp </allcaps> - <allcaps> covid </allcaps> investigates health impacts of <hashtag> covid 19 </hashtag> on hospitalised patients including <hashtag> mental health </hashtag> & neurological problems . find out more <url> <user> <url>


In [11]:
# Define BERT tokenizer
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [12]:
# Tokenize train and validation data
train_enc = tokenizer.batch_encode_plus(train.clean_tweets.to_list(), padding="longest", truncation=True, max_length=128, return_tensors="pt")
valid_enc = tokenizer.batch_encode_plus(validation.clean_tweets.to_list(), padding="longest", truncation=True, max_length=128, return_tensors="pt")

In [13]:
train_enc.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [14]:
train_enc.input_ids.shape, train_enc.token_type_ids.shape, train_enc.attention_mask.shape

(torch.Size([9067, 128]), torch.Size([9067, 128]), torch.Size([9067, 128]))

In [15]:
BATCH_SIZE = 32
N_EPOCHS = 5

In [16]:
def get_dataloader(encoding, target):
    data = (TensorDataset(encoding.input_ids, encoding.token_type_ids, encoding.attention_mask, target))
    sampler = RandomSampler(data)
    dataloader = DataLoader(data, sampler=sampler, batch_size=BATCH_SIZE)
    return dataloader

In [17]:
# Transform the target variable

le = LabelEncoder()
train.label = le.fit_transform(train.label)
validation.label = le.transform(validation.label)

In [18]:
list(le.classes_)

['Lit-News_mentions', 'Nonpersonal_reports', 'Self_reports']

In [19]:
train_dataloader = get_dataloader(train_enc, torch.tensor(train['label'].to_list()))
valid_dataloader = get_dataloader(valid_enc, torch.tensor(validation['label'].to_list()))

In [20]:
# Sanity check that the tensors returned by the dataloader are correct
for batch in train_dataloader:
    input_ids, type_ids, attn_mask, target = batch
    print(input_ids.shape, type_ids.shape, attn_mask.shape, target.shape)
    break

torch.Size([32, 128]) torch.Size([32, 128]) torch.Size([32, 128]) torch.Size([32])


## 3. Model Building - BERT

In [21]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [22]:
class BERTclassifier(nn.Module):
    def __init__(self, transformer):
        super(BERTclassifier, self).__init__()
        self.transformer = transformer
        self.linear_layer = nn.Linear(768, 3)
    
    def forward(self, ip_ids, type_ids, attn_mask):
        op = self.transformer(input_ids=ip_ids,
                              attention_mask=attn_mask, 
                              token_type_ids=type_ids)
        return  self.linear_layer(op["pooler_output"])

In [23]:
def count_parameter(model):
    return sum(para.numel() for para in model.parameters() if para.requires_grad)

In [24]:
transformer = BertModel.from_pretrained("bert-base-uncased")
model = BERTclassifier(transformer).to(device)
print(f"The model has {count_parameter(model):,} trainable parameters.")

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


The model has 109,484,547 trainable parameters.


In [25]:
# for name, param in model.named_parameters():
#     if "pooler" in name or "linear" in name:#or "layer.11" in name or "layer.10" in name or "linear" in name:
#         param.requires_grad = True
#     else:
#         param.requires_grad = False
#   print(name, param.shape, param.requires_grad)

# print(f'The model has {count_parameter(model):,} trainable parameters')

In [26]:
# Define optimizer and 
criterion = torch.nn.CrossEntropyLoss()
optim = torch.optim.AdamW(model.parameters(), lr = 2e-5)

#### Train BERT Model

In [27]:
def train_model(model, dataloader, clip=1.0):
    model.train()

    epoch_loss = 0
    batch_num = 0
    pred, target = [], []

    for index, batch in tqdm(enumerate(dataloader)):
        batch = tuple(row.to(device) for row in batch)
        input_ids, type_ids, attn_mask, y = batch

        optim.zero_grad()
        output = model(input_ids, type_ids, attn_mask)
        loss = criterion(output, y)
        loss.backward()

        nn.utils.clip_grad_norm_(model.parameters(), clip)
        optim.step()

        epoch_loss += loss.item()
        batch_num += 1
        pred.extend(torch.argmax(output, -1).tolist())
        target.extend(y.tolist())
    
    return epoch_loss/batch_num, f1_score(target, pred, average='micro')

def evaluate(model, dataloader):
    model.eval()

    epoch_loss = 0
    batch_num = 0
    pred, target = list(), list()

    for index, batch in enumerate(dataloader):
        batch = tuple(row.to(device) for row in batch)
        input_ids, type_ids, attn_mask, y = batch
        
        with torch.no_grad():
            output = model(input_ids, type_ids, attn_mask)
            loss = criterion(output, y)
            
            epoch_loss += loss.item()
            batch_num += 1
            pred.extend(torch.argmax(output, -1).tolist())
            target.extend(y.tolist())
    
    return epoch_loss/batch_num, f1_score(target, pred, average='micro'), pred, target

In [28]:
best_valid_loss = float('inf')
total_train_loss, total_valid_loss = list(), list()

In [29]:
for epoch in tqdm(range(N_EPOCHS)):
    train_loss, train_f1_score = train_model(model, train_dataloader)
    total_train_loss.append(train_loss)

    valid_loss, valid_f1_score, pred, target = evaluate(model, valid_dataloader)
    total_valid_loss.append(valid_loss)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        best_pred, best_target = pred, target
        torch.save(model.state_dict(), "model_least_loss.pt")
        print("Best Model Saved!!\n")
    elif epoch % 3 == 0:
        torch.save(model.state_dict(), "model_checkpoint" + str(epoch) + ".pt")
        print("Checkpoint Model Saved!\n")
    print(f"Epoch: {epoch+1:02}")
    print(f"Train Total Loss: {train_loss:.3f} | Train F1 Score: {train_f1_score:.3f}")
    print(f"Valid Total Loss: {valid_loss:.3f} | Valid F1 Score: {valid_f1_score:.3f}")
    print("-"*20)

  0%|          | 0/5 [00:00<?, ?it/s]
0it [00:00, ?it/s]
1it [00:01,  1.88s/it]
2it [00:03,  1.75s/it]
3it [00:05,  1.71s/it]
4it [00:06,  1.70s/it]
5it [00:08,  1.71s/it]
6it [00:10,  1.70s/it]
7it [00:11,  1.70s/it]
8it [00:13,  1.69s/it]
9it [00:15,  1.69s/it]
10it [00:17,  1.68s/it]
11it [00:18,  1.68s/it]
12it [00:20,  1.68s/it]
13it [00:22,  1.68s/it]
14it [00:23,  1.69s/it]
15it [00:25,  1.68s/it]
16it [00:27,  1.68s/it]
17it [00:28,  1.67s/it]
18it [00:30,  1.66s/it]
19it [00:32,  1.65s/it]
20it [00:33,  1.65s/it]
21it [00:35,  1.65s/it]
22it [00:36,  1.65s/it]
23it [00:38,  1.65s/it]
24it [00:40,  1.65s/it]
25it [00:41,  1.64s/it]
26it [00:43,  1.64s/it]
27it [00:45,  1.65s/it]
28it [00:46,  1.65s/it]
29it [00:48,  1.65s/it]
30it [00:50,  1.65s/it]
31it [00:51,  1.65s/it]
32it [00:53,  1.65s/it]
33it [00:55,  1.65s/it]
34it [00:56,  1.65s/it]
35it [00:58,  1.65s/it]
36it [01:00,  1.65s/it]
37it [01:01,  1.65s/it]
38it [01:03,  1.65s/it]
39it [01:04,  1.65s/it]
40it [01:06,  1.

Best Model Saved!!

Epoch: 01
Train Total Loss: 0.223 | Train F1 Score: 0.933
Valid Total Loss: 0.078 | Valid F1 Score: 0.970
--------------------



0it [00:00, ?it/s]
1it [00:01,  1.64s/it]
2it [00:03,  1.64s/it]
3it [00:04,  1.64s/it]
4it [00:06,  1.65s/it]
5it [00:08,  1.65s/it]
6it [00:09,  1.65s/it]
7it [00:11,  1.64s/it]
8it [00:13,  1.64s/it]
9it [00:14,  1.64s/it]
10it [00:16,  1.64s/it]
11it [00:18,  1.64s/it]
12it [00:19,  1.64s/it]
13it [00:21,  1.64s/it]
14it [00:23,  1.64s/it]
15it [00:24,  1.64s/it]
16it [00:26,  1.64s/it]
17it [00:27,  1.64s/it]
18it [00:29,  1.64s/it]
19it [00:31,  1.64s/it]
20it [00:32,  1.64s/it]
21it [00:34,  1.64s/it]
22it [00:36,  1.64s/it]
23it [00:37,  1.64s/it]
24it [00:39,  1.64s/it]
25it [00:41,  1.64s/it]
26it [00:42,  1.64s/it]
27it [00:44,  1.64s/it]
28it [00:46,  1.64s/it]
29it [00:47,  1.64s/it]
30it [00:49,  1.64s/it]
31it [00:50,  1.64s/it]
32it [00:52,  1.64s/it]
33it [00:54,  1.64s/it]
34it [00:55,  1.64s/it]
35it [00:57,  1.64s/it]
36it [00:59,  1.64s/it]
37it [01:00,  1.64s/it]
38it [01:02,  1.64s/it]
39it [01:04,  1.64s/it]
40it [01:05,  1.64s/it]
41it [01:07,  1.64s/it]
42it 

Best Model Saved!!

Epoch: 02
Train Total Loss: 0.102 | Train F1 Score: 0.970
Valid Total Loss: 0.060 | Valid F1 Score: 0.980
--------------------



0it [00:00, ?it/s]
1it [00:01,  1.63s/it]
2it [00:03,  1.64s/it]
3it [00:04,  1.64s/it]
4it [00:06,  1.64s/it]
5it [00:08,  1.64s/it]
6it [00:09,  1.65s/it]
7it [00:11,  1.65s/it]
8it [00:13,  1.65s/it]
9it [00:14,  1.65s/it]
10it [00:16,  1.64s/it]
11it [00:18,  1.64s/it]
12it [00:19,  1.64s/it]
13it [00:21,  1.64s/it]
14it [00:23,  1.64s/it]
15it [00:24,  1.64s/it]
16it [00:26,  1.64s/it]
17it [00:27,  1.64s/it]
18it [00:29,  1.64s/it]
19it [00:31,  1.64s/it]
20it [00:32,  1.64s/it]
21it [00:34,  1.65s/it]
22it [00:36,  1.65s/it]
23it [00:37,  1.65s/it]
24it [00:39,  1.65s/it]
25it [00:41,  1.65s/it]
26it [00:42,  1.65s/it]
27it [00:44,  1.65s/it]
28it [00:46,  1.65s/it]
29it [00:47,  1.65s/it]
30it [00:49,  1.65s/it]
31it [00:50,  1.65s/it]
32it [00:52,  1.65s/it]
33it [00:54,  1.65s/it]
34it [00:55,  1.65s/it]
35it [00:57,  1.65s/it]
36it [00:59,  1.65s/it]
37it [01:00,  1.65s/it]
38it [01:02,  1.65s/it]
39it [01:04,  1.65s/it]
40it [01:05,  1.65s/it]
41it [01:07,  1.64s/it]
42it 

Epoch: 03
Train Total Loss: 0.067 | Train F1 Score: 0.982
Valid Total Loss: 0.064 | Valid F1 Score: 0.978
--------------------



0it [00:00, ?it/s]
1it [00:01,  1.64s/it]
2it [00:03,  1.64s/it]
3it [00:04,  1.64s/it]
4it [00:06,  1.64s/it]
5it [00:08,  1.64s/it]
6it [00:09,  1.64s/it]
7it [00:11,  1.64s/it]
8it [00:13,  1.64s/it]
9it [00:14,  1.64s/it]
10it [00:16,  1.64s/it]
11it [00:18,  1.64s/it]
12it [00:19,  1.64s/it]
13it [00:21,  1.64s/it]
14it [00:23,  1.64s/it]
15it [00:24,  1.64s/it]
16it [00:26,  1.64s/it]
17it [00:27,  1.64s/it]
18it [00:29,  1.64s/it]
19it [00:31,  1.64s/it]
20it [00:32,  1.64s/it]
21it [00:34,  1.64s/it]
22it [00:36,  1.64s/it]
23it [00:37,  1.64s/it]
24it [00:39,  1.64s/it]
25it [00:41,  1.65s/it]
26it [00:42,  1.64s/it]
27it [00:44,  1.64s/it]
28it [00:46,  1.64s/it]
29it [00:47,  1.64s/it]
30it [00:49,  1.64s/it]
31it [00:50,  1.64s/it]
32it [00:52,  1.64s/it]
33it [00:54,  1.64s/it]
34it [00:55,  1.64s/it]
35it [00:57,  1.64s/it]
36it [00:59,  1.64s/it]
37it [01:00,  1.64s/it]
38it [01:02,  1.64s/it]
39it [01:04,  1.64s/it]
40it [01:05,  1.64s/it]
41it [01:07,  1.64s/it]
42it 

Checkpoint Model Saved!

Epoch: 04
Train Total Loss: 0.038 | Train F1 Score: 0.989
Valid Total Loss: 0.067 | Valid F1 Score: 0.984
--------------------



0it [00:00, ?it/s]
1it [00:01,  1.65s/it]
2it [00:03,  1.66s/it]
3it [00:04,  1.65s/it]
4it [00:06,  1.66s/it]
5it [00:08,  1.66s/it]
6it [00:09,  1.66s/it]
7it [00:11,  1.66s/it]
8it [00:13,  1.66s/it]
9it [00:14,  1.66s/it]
10it [00:16,  1.66s/it]
11it [00:18,  1.66s/it]
12it [00:19,  1.66s/it]
13it [00:21,  1.66s/it]
14it [00:23,  1.66s/it]
15it [00:24,  1.66s/it]
16it [00:26,  1.66s/it]
17it [00:28,  1.66s/it]
18it [00:29,  1.66s/it]
19it [00:31,  1.66s/it]
20it [00:33,  1.66s/it]
21it [00:34,  1.66s/it]
22it [00:36,  1.66s/it]
23it [00:38,  1.66s/it]
24it [00:39,  1.66s/it]
25it [00:41,  1.66s/it]
26it [00:43,  1.66s/it]
27it [00:44,  1.66s/it]
28it [00:46,  1.66s/it]
29it [00:48,  1.66s/it]
30it [00:49,  1.66s/it]
31it [00:51,  1.66s/it]
32it [00:53,  1.66s/it]
33it [00:54,  1.66s/it]
34it [00:56,  1.66s/it]
35it [00:58,  1.66s/it]
36it [00:59,  1.66s/it]
37it [01:01,  1.66s/it]
38it [01:03,  1.66s/it]
39it [01:04,  1.66s/it]
40it [01:06,  1.66s/it]
41it [01:08,  1.66s/it]
42it 

Epoch: 05
Train Total Loss: 0.025 | Train F1 Score: 0.994
Valid Total Loss: 0.063 | Valid F1 Score: 0.986
--------------------


In [31]:
print(classification_report(best_target, best_pred))

              precision    recall  f1-score   support

           0       0.99      0.98      0.99       247
           1       0.97      0.97      0.97       180
           2       0.97      0.99      0.98        73

    accuracy                           0.98       500
   macro avg       0.98      0.98      0.98       500
weighted avg       0.98      0.98      0.98       500



In [30]:
f1_score(best_target, best_pred, average='micro')

0.98